In [2]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('SparkML-example').getOrCreate()
spark

Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/11/20 11:46:21 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
iris_filepath = "/home/ubuntu/working/spark-examples/data/iris.csv"

iris_sdf = spark.read.csv(f"file://{iris_filepath}", inferSchema=True, header=True)
iris_sdf.show(5)

+------------+-----------+------------+-----------+------+
|sepal_length|sepal_width|petal_length|petal_width|target|
+------------+-----------+------------+-----------+------+
|         5.1|        3.5|         1.4|        0.2|     0|
|         4.9|        3.0|         1.4|        0.2|     0|
|         4.7|        3.2|         1.3|        0.2|     0|
|         4.6|        3.1|         1.5|        0.2|     0|
|         5.0|        3.6|         1.4|        0.2|     0|
+------------+-----------+------------+-----------+------+
only showing top 5 rows



# LogisticRegression 사용하기

In [4]:
from pyspark.ml.classification import LogisticRegression

# 훈련 세트 변환
train_sdf, test_sdf = iris_sdf.randomSplit([0.8, 0.2], seed=42)

In [5]:
train_sdf.cache()

DataFrame[sepal_length: double, sepal_width: double, petal_length: double, petal_width: double, target: int]

In [6]:
# VectorAssembler
from pyspark.ml.feature import VectorAssembler

iris_columns = ["sepal_length", "sepal_width", "petal_length", "petal_width"]
vec_assembler = VectorAssembler(inputCols=iris_columns, outputCol='features')

In [7]:
vec_train_sdf = vec_assembler.transform(train_sdf)
vec_train_sdf.show(5)

+------------+-----------+------------+-----------+------+-----------------+
|sepal_length|sepal_width|petal_length|petal_width|target|         features|
+------------+-----------+------------+-----------+------+-----------------+
|         4.3|        3.0|         1.1|        0.1|     0|[4.3,3.0,1.1,0.1]|
|         4.4|        2.9|         1.4|        0.2|     0|[4.4,2.9,1.4,0.2]|
|         4.4|        3.2|         1.3|        0.2|     0|[4.4,3.2,1.3,0.2]|
|         4.5|        2.3|         1.3|        0.3|     0|[4.5,2.3,1.3,0.3]|
|         4.6|        3.1|         1.5|        0.2|     0|[4.6,3.1,1.5,0.2]|
+------------+-----------+------------+-----------+------+-----------------+
only showing top 5 rows



In [8]:
# 모델 훈련
from pyspark.ml.classification import LogisticRegression

lr = LogisticRegression(
    featuresCol='features',
    labelCol='target'
)

In [9]:
lr_model = lr.fit(vec_train_sdf)

23/11/20 11:57:35 WARN InstanceBuilder$NativeBLAS: Failed to load implementation from:dev.ludovic.netlib.blas.JNIBLAS
23/11/20 11:57:35 WARN InstanceBuilder$NativeBLAS: Failed to load implementation from:dev.ludovic.netlib.blas.ForeignLinkerBLAS


In [10]:
vec_test_sdf = vec_assembler.transform(test_sdf)
vec_test_sdf.show(5)

+------------+-----------+------------+-----------+------+-----------------+
|sepal_length|sepal_width|petal_length|petal_width|target|         features|
+------------+-----------+------------+-----------+------+-----------------+
|         4.4|        3.0|         1.3|        0.2|     0|[4.4,3.0,1.3,0.2]|
|         4.6|        3.2|         1.4|        0.2|     0|[4.6,3.2,1.4,0.2]|
|         4.6|        3.6|         1.0|        0.2|     0|[4.6,3.6,1.0,0.2]|
|         4.8|        3.1|         1.6|        0.2|     0|[4.8,3.1,1.6,0.2]|
|         4.9|        3.1|         1.5|        0.1|     0|[4.9,3.1,1.5,0.1]|
+------------+-----------+------------+-----------+------+-----------------+
only showing top 5 rows



In [11]:
# 테스트 세트 예측

predictions = lr_model.transform(vec_test_sdf)
predictions.show(5)

+------------+-----------+------------+-----------+------+-----------------+--------------------+--------------------+----------+
|sepal_length|sepal_width|petal_length|petal_width|target|         features|       rawPrediction|         probability|prediction|
+------------+-----------+------------+-----------+------+-----------------+--------------------+--------------------+----------+
|         4.4|        3.0|         1.3|        0.2|     0|[4.4,3.0,1.3,0.2]|[54.8316731860621...|[1.0,5.8045981223...|       0.0|
|         4.6|        3.2|         1.4|        0.2|     0|[4.6,3.2,1.4,0.2]|[59.0189684197576...|[1.0,1.7180006172...|       0.0|
|         4.6|        3.6|         1.0|        0.2|     0|[4.6,3.6,1.0,0.2]|[76.5905029625326...|[1.0,1.3749820140...|       0.0|
|         4.8|        3.1|         1.6|        0.2|     0|[4.8,3.1,1.6,0.2]|[52.2432095666216...|[1.0,1.6438217837...|       0.0|
|         4.9|        3.1|         1.5|        0.1|     0|[4.9,3.1,1.5,0.1]|[55.1241026366

In [14]:
# 모델 평가
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

evaluator_accuracy = MulticlassClassificationEvaluator(
    labelCol = 'target',
    predictionCol = 'prediction',
    metricName = 'accuracy'
)

In [15]:
accuracy = evaluator_accuracy.evaluate(predictions)
accuracy

1.0

In [16]:
spark.stop()